In [5]:
#!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd 
import numpy as np

In [3]:
dep, num, nom_voie, ville, code_postal = "75", 36, "Rue Chardon-Lagache", "Paris 16e Arrondissement", "75016"

In [4]:
#DataFrame regroupant des informations sur les centrales nucléaires de France métropolitaine 
centrales = pd.read_csv("DataSets/centrales_nucleaires.csv",sep=',',encoding_errors='ignore')

In [5]:
#Calcul de la plus petite distance à une centrale nucléaire 
def min_dist_centrales(longitude,latitude, centrales) :
    long, lat = conversion_radians(longitude,latitude) #conversion des coordonnees en radians 
    d = np.inf
    k = 0
    for i in range(14) : #il y a 14 centrales nucleaires en FR 
        long_centrale, lat_centrale = conversion_radians(centrales["Commune long"][i], centrales["Commune Lat"][i])
        D = 6371 * np.arccos(np.sin(lat_centrale)*np.sin(lat) + np.cos(lat_centrale)*np.cos(lat)*np.cos(long_centrale - long))
        if D<d :
            d = D 
            k = i
    return(d,centrales["Centrale nuclaire"][k])

In [6]:
#Calcul du risque nucléaire 
def risque_nucleaire(longitude,latitude):
    d_min, r = min_dist_centrales(longitude,latitude)
    if d_min <= 5 :
        return(5)
    elif d_min <= 20 :
        return(4)
    elif d_min <= 60 :
        return(3)
    elif d_min <= 100 :
        return(2)
    else :
        return(1) 

In [7]:
#Conversion des coordonnees GPS en radian 
def conversion_radians(longitude,latitude):
    GPS_rad = ((longitude*np.pi)/180 , (latitude*np.pi)/180)
    return(GPS_rad)

In [10]:
def nucleaire(dep,num,nom_voie,ville,code_postal):

    adresses_dep = pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-"+dep+".csv.gz", compression='gzip', sep=";", error_bad_lines=False) #recuperation des adresses postales correspondant au département de l'utilisateur
    
    foyer_infos = adresses_dep[(adresses_dep["numero"] == num) & (adresses_dep["nom_commune"] == ville) & (adresses_dep["nom_voie"] == nom_voie)] 
    foyer = foyer_infos.to_numpy()
    foyer = foyer.tolist()
    print(foyer) #informations sur le domicile de l'utilisateur

    #Coordonnees geographiques du dommicile 
    longitude = foyer[0][12]
    latitude = foyer[0][13]

    #Centrale nucléaire la plus proche, distance associé 
    D, nearest_centrale = min_dist_centrales(longitude,latitude)
    print("La centrale nucleaire la plus proche se situe à :")
    print(D,nearest_centrale)

    #Mesure du risque nucléaire à cette adresse 
    R = risque_nucleaire(longitude,latitude)
    print("Le risque nucléaire à cette adresse est de :")

    return(R)



In [11]:
#Sortie du code :
print(nucleaire(dep, num, nom_voie, ville, code_postal))

TypeError: read_csv() got an unexpected keyword argument 'verify'